An interactive introduction to Noodles
======================================

Noodles is there to make your life easier, *in parallel*! The reason why Noodles can be easy and do parallel Python at the same time is its *functional* approach. In one part you'll define a set of functions that you'd like to run with Noodles, in an other part you'll compose these functions into a *workflow graph*. To make this approach work a function should not have any *side effects*. Let's not linger and just start noodling! First we define some functions to use.

In [1]:
from noodles import schedule

@schedule
def add(x, y):
    return x + y

@schedule
def mul(x,y):
    return x * y

Now we can create a workflow composing several calls to this function.

In [2]:
a = add(1, 1)
b = mul(a, 2)
c = add(a, a)
d = mul(b, c)

That looks easy enough; the funny thing is though, that nothing has been computed yet! Noodles just created the workflow graphs corresponding to the values that still need to be computed. Until such time, we work with the *promise* of a future value. Using some function in `pygraphviz` we can look at the call graphs.

In [3]:
from draw_workflow import draw_workflow
import sys
import os

draw_workflow("wf1a.png", a._workflow)
draw_workflow("wf1b.png", b._workflow)
draw_workflow("wf1c.png", c._workflow)
draw_workflow("wf1d.png", d._workflow)

err = os.system("montage wf1?.png -tile 4x1 -geometry +10+0 wf1-series.png")

![callgraph](wf1-series.png)
Now, to compute the result we have to tell Noodles to evaluate the program.

In [12]:
from noodles import run_parallel

run_parallel(d, n_threads=2)

16

## Making loops

Thats all swell, but how do we make a parallel loop? Let's look at a `map` operation; in Python there are several ways to perform a function on all elements in an array. For this example, we will translate some words using the Glosbe service, which has a nice REST interface. We first build some functionality to use this interface.

In [44]:
import urllib.request
import json
import re


class Translate:
    """Translate words and sentences in the worst possible way. The Glosbe dictionary
    has a nice REST interface that we query for a phrase. We then take the first result.
    To translate a sentence, we cut it in pieces, translate it and paste it back into
    a Frankenstein monster."""
    def __init__(self, src_lang='en', tgt_lang='fy'):
        self.src = src_lang
        self.tgt = tgt_lang
        self.url = 'https://glosbe.com/gapi/translate?' \
                   'from={src}&dest={tgt}&' \
                   'phrase={{phrase}}&format=json'.format(
                        src=src_lang, tgt=tgt_lang)
    
    def query_phrase(self, phrase):
        with urllib.request.urlopen(self.url.format(phrase=phrase.lower())) as response:
            translation = json.loads(response.read().decode())
        return translation

    def word(self, phrase):
        translation = self.query_phrase(phrase)
        
        if len(translation['tuc']) > 0 and 'phrase' in translation['tuc'][0]:
            result = translation['tuc'][0]['phrase']['text']
            if phrase[0].isupper():
                return result.title()
            else:
                return result            
        else:
            return "<" + phrase + ">"
    
    def sentence(self, phrase):
        words = re.sub("[^\w]", " ", phrase).split()
        space = re.sub("[\w]+", "{}", phrase)
        return space.format(*(self.word(w) for w in words))

We start with a list of strings that desparately need translation.

In [5]:
shakespeare = [
    "If music be the food of love, play on,",
    "Give me excess of it; that surfeiting,",
    "The appetite may sicken, and so die."]

def print_poem(intro, poem):
    print(intro)
    for line in poem:
        print("     ", line)
    print()

print_poem("Original:", shakespeare)

Original:
      If music be the food of love, play on,
      Give me excess of it; that surfeiting,
      The appetite may sicken, and so die.



Beginning Python programmers like to append things; this is not how you are
supposed to program in Python; if you do, please go and read Jeff Knupp's *Writing Idiomatic Python*.

In [6]:
shakespeare_auf_deutsch = []
for line in shakespeare:
    shakespeare_auf_deutsch.append(
        Translate('en', 'de').sentence(line))
print_poem("Auf Deutsch:", shakespeare_auf_deutsch)

Auf Deutsch:
      Wenn Musik sein der Essen von Minne, spielen an,
      Geben ich Übermaß von es; das übersättigend,
      Der Appetit dürfen Ekel erregen, und so sterben.



Rather use a comprehension like so:

In [7]:
shakespeare_ynt_frysk = \
    (Translate('en', 'fy').sentence(line) for line in shakespeare)
print_poem("Yn it Frysk:", shakespeare_ynt_frysk)

Yn it Frysk:
      At muzyk wêze de fiedsel fan leafde, boartsje oan,
      Jaan <me> by fersin fan it; dat <surfeiting>,
      De <appetite> maaie <sicken>, en dus deagean.



Or use `map`:

In [8]:
shakespeare_pa_dansk = \
    map(Translate('en', 'da').sentence, shakespeare)
print_poem("På Dansk:", shakespeare_pa_dansk)

På Dansk:
      Hvis musik være de mad af kærlighed, spil på,
      Give mig udskejelser af det; som <surfeiting>,
      De appetit må <sicken>, og så dø.



If your connection is a bit slow, you may find that the translations take a while to process. Wouldn't it be nice to do it in parallel? How much code would we have to change to get there in Noodles? Let's take the slow part of the program and add a `@schedule` decorator, and run! Sadly, it is not that simple. We can add `@schedule` to the `word` method. This means that it will return a promise. 

* Rule: *Functions that take promises need to be scheduled functions, or refer to a scheduled function at some level.* 

We could write

    return schedule(space.format)(*(self.word(w) for w in words))
    
in the last line of the `sentence` method, but the string format method doesn't support wrapping. We rely on getting the signature of a function by calling `inspect.signature`. In some cases of build-in function this raises an exception. We may find a work around for these cases in future versions of Noodles. For the moment we'll have to define a little wrapper function.

In [45]:
from noodles import schedule


@schedule
def format_string(s, *args, **kwargs):
    return s.format(*args, **kwargs)


import urllib.request
import json
import re


class Translate:
    """Translate words and sentences in the worst possible way. The Glosbe dictionary
    has a nice REST interface that we query for a phrase. We then take the first result.
    To translate a sentence, we cut it in pieces, translate it and paste it back into
    a Frankenstein monster."""
    def __init__(self, src_lang='en', tgt_lang='fy'):
        self.src = src_lang
        self.tgt = tgt_lang
        self.url = 'https://glosbe.com/gapi/translate?' \
                   'from={src}&dest={tgt}&' \
                   'phrase={{phrase}}&format=json'.format(
                        src=src_lang, tgt=tgt_lang)
    
    def query_phrase(self, phrase):
        with urllib.request.urlopen(self.url.format(phrase=phrase.lower())) as response:
            translation = json.loads(response.read().decode())
        return translation
    
    @schedule
    def word(self, phrase):
        with urllib.request.urlopen(self.url.format(phrase=phrase.lower())) as response:
            translation = json.loads(response.read().decode())
        
        if len(translation['tuc']) > 0 and 'phrase' in translation['tuc'][0]:
            result = translation['tuc'][0]['phrase']['text']
            if phrase[0].isupper():
                return result.title()
            else:
                return result            
        else:
            return "<" + phrase + ">"
        
    def sentence(self, phrase):
        words = re.sub("[^\w]", " ", phrase).split()
        space = re.sub("[\w]+", "{}", phrase)
        return format_string(space, *(self.word(w) for w in words))
    
    def __str__(self):
        return "[{} -> {}]".format(self.src, self.tgt)

Let's take stock of the mutations to the original. We've added a `@schedule` decorator to `word`, and changed a function call in `sentence`.

In [49]:
import difflib
from pprint import pprint

d = difflib.Differ()
result = difflib.unified_diff(
    [a.rstrip() for a in In[44].split('\n')], 
    [a.rstrip() for a in In[45].split('\n')])
print('\n'.join(result))

--- 

+++ 

@@ -1,3 +1,11 @@

+from noodles import schedule
+
+
+@schedule
+def format_string(s, *args, **kwargs):
+    return s.format(*args, **kwargs)
+
+
 import urllib.request
 import json
 import re
@@ -19,6 +27,7 @@

             translation = json.loads(response.read().decode())
         return translation
 
+    @schedule
     def word(self, phrase):
         with urllib.request.urlopen(self.url.format(phrase=phrase.lower())) as response:
             translation = json.loads(response.read().decode())
@@ -35,7 +44,7 @@

     def sentence(self, phrase):
         words = re.sub("[^\w]", " ", phrase).split()
         space = re.sub("[\w]+", "{}", phrase)
-        return space.format(*(self.word(w) for w in words))
+        return format_string(space, *(self.word(w) for w in words))
 
     def __str__(self):
         return "[{} -> {}]".format(self.src, self.tgt)


In [51]:
from noodles import gather

shakespeare_en_esperanto = \
    map(Translate('en', 'eo').sentence, shakespeare)

wf = gather(*shakespeare_en_esperanto)
draw_workflow('poetry.png', wf._workflow)
result = run_parallel(wf, n_threads=12)
print_poem("Shakespeare en Esperanto:", result)

Shakespeare en Esperanto:
      Se muziko esti la manĝaĵo de ami, ludi sur,
      Doni mi eksceso de ĝi; ke <surfeiting>,
      La apetito povi naŭzi, kaj tiel morti.



The workflow graph of this script looks like this.

![callgraph](poetry.png)